## IMPORT LIBRARIES

In [72]:
import time
start = time.time()

In [73]:
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix

## `Length` influence test

### Dataset - Exclude Length

In [74]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

exclude `length` features.

Including prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words, unstemmed, stemmed, POS, POS/total_words

In [75]:
X = dataset.iloc[:,10:15].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[4.17457306e-01, 1.12000000e+02, 2.12523719e-01, 5.84000000e+02,
        5.59000000e+02],
       [4.55555556e-01, 6.60000000e+01, 3.66666667e-01, 2.10000000e+02,
        2.10000000e+02],
       [5.51724138e-01, 8.30000000e+01, 3.18007663e-01, 2.91000000e+02,
        2.85000000e+02],
       ...,
       [5.37878788e-01, 3.40000000e+01, 2.57575758e-01, 1.64000000e+02,
        1.60000000e+02],
       [5.36541889e-01, 1.68000000e+02, 2.99465241e-01, 5.42000000e+02,
        5.12000000e+02],
       [4.44230769e-01, 1.65000000e+02, 3.17307692e-01, 5.14000000e+02,
        4.96000000e+02]])

In [76]:
X.shape

(1800, 5)

In [77]:
y = np.array(y).reshape(-1,1)
y.shape

(1800, 1)

In [78]:
### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


## Model Training excluding `Length` features

### Naive Bayes Training 

In [79]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [80]:
X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

In [81]:
model_nbl = naive_bayes.MultinomialNB()
model_nbl.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

The Naive Bayes model excluding `length` features is called `model_nbl`

### SVM Training using `Length` features only

In [154]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)

In [83]:
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svml = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svml.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

The Support Vector Machine (SVM) model excluding `length` features is now called `model_svml`

### BLRR


In [84]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

In [85]:
from sklearn import linear_model
model_blrrl = linear_model.BayesianRidge()
model_blrrl.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

At this stage, the Bayesian Linear Ridge Regression (BLRR) model excluding `length` feature is called `model_blrrl`


## Prediction excluding `length` feature 

### Naive Bayes excluding `length` features

In [86]:
y_predNBL = model_nbl.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()
#cm = confusion_matrix(y_test, y_predNBL)
#print(cm)

### SVM Excluding `length` 

In [87]:
y_predSVML = model_svml.predict(X_testSVM)
y_predSVML = sc_ysvm.inverse_transform(y_predSVML).round()

#cm = confusion_matrix(y_test, y_predSVML)
#np.set_printoptions(threshold=np.inf)
#print(cm)

### BLRR excluding `length` 

In [88]:
y_predBLRRL = model_blrrl.predict(X_testBLRR)
y_predBLRRL = sc_yblrr.inverse_transform(y_predBLRRL).round()

#cm = confusion_matrix(y_test, y_predBLRRL)
#print(cm)

## Ensembling the 3 Algo excluding `length` features

In [89]:
actual = pd.Series(y_test.ravel())
predNBL = pd.Series(y_predNBL)
predSVML = pd.Series(y_predSVML)
predBLRRL = pd.Series(y_predBLRRL)

data = {"Actual": actual,
        "NBL": predNBL, 
        "SVML": predSVML, 
        "BLRRL": predBLRRL} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NBL,SVML,BLRRL
0,3.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0
3,4.0,4.0,3.0,3.0
4,4.0,5.0,4.0,4.0


In [90]:
results['EnsembleL'] = np.where(
                            (results['NBL'] == results['BLRRL']) |
                            (results['NBL'] == results['SVML']),
                            results['NBL'],
                            results['BLRRL']
                        )
results.head(5)

,Actual,NBL,SVML,BLRRL,EnsembleL
0,3.0,4.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,4.0,3.0,3.0,3.0
4,4.0,5.0,4.0,4.0,4.0


## Evaluation using QWK

In [91]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

### Naive Bayes

In [92]:
rpt = classification_report(y_test, y_predNBL)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.00      0.00      0.00        23
         3.0       0.61      0.59      0.60       160
         4.0       0.59      0.74      0.66       156
         5.0       0.12      0.08      0.10        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.59       360
   macro avg       0.22      0.23      0.22       360
weighted avg       0.53      0.59      0.55       360



C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
print(cohen_kappa_score(y_test, y_predNBL, weights="quadratic"))

0.44386909693454846


### SVM 

In [94]:
rpt = classification_report(y_test, y_predSVML)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.60      0.39      0.47        23
         3.0       0.64      0.74      0.68       160
         4.0       0.65      0.67      0.66       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.31      0.30      0.30       360
weighted avg       0.60      0.64      0.62       360



In [95]:
print(cohen_kappa_score(y_test, y_predSVML, weights="quadratic"))

0.564516129032258


### BLRR

In [96]:
rpt = classification_report(y_test, y_predBLRRL)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.42      0.35      0.38        23
         3.0       0.63      0.76      0.69       160
         4.0       0.67      0.63      0.65       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.63       360
   macro avg       0.29      0.29      0.29       360
weighted avg       0.60      0.63      0.61       360



In [97]:
print(cohen_kappa_score(y_test, y_predBLRRL, weights="quadratic"))

0.6013422818791946


### Ensemble

In [98]:
rpt = classification_report(y_test,results['EnsembleL'])
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.54      0.30      0.39        23
         3.0       0.64      0.76      0.69       160
         4.0       0.66      0.66      0.66       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.31      0.29      0.29       360
weighted avg       0.60      0.64      0.62       360



In [99]:
print(cohen_kappa_score(y_test, results['EnsembleL'], weights="quadratic"))

0.561464569121537


#### QWK Scores result excluding `length` features only:

Naive Bayes: 0.44386909693454846

SVM: 0.564516129032258

BLRR: 0.6013422818791946

Ensemble: 0.561464569121537


## `POS` influence test 

In [100]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [101]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559,4
1,1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210,1
2,2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285,2
3,3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528,4
4,4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,1339.0,286.0,0.0,34.0,11.0,4.681818,31.891253,0.111508,114.0,0.398601,78.0,0.272727,303,293,3
1796,1796,1373.0,282.0,15.0,8.0,15.0,4.868794,12.757794,0.045240,152.0,0.539007,80.0,0.283688,339,333,3
1797,1797,605.0,132.0,7.0,2.0,5.0,4.583333,3.387097,0.025660,71.0,0.537879,34.0,0.257576,164,160,2
1798,1798,2737.0,561.0,22.0,5.0,27.0,4.878788,20.071942,0.035779,301.0,0.536542,168.0,0.299465,542,512,3


exclude only `PoS` features.

Including chars, words, commas, apostrophes, punctuations, avg_word_length, prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words, unstemmed, stemmed

In [102]:
X = dataset.iloc[:,[1,2,3,4,5,6,9,10,11,12,13,14]].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X[0]

array([2.63900000e+03, 5.27000000e+02, 1.50000000e+01, 1.30000000e+01,
       2.10000000e+01, 5.00759013e+00, 2.20000000e+02, 4.17457306e-01,
       1.12000000e+02, 2.12523719e-01, 5.84000000e+02, 5.59000000e+02])

In [103]:
X.shape

(1800, 12)

In [104]:
y = np.array(y).reshape(-1,1)
y.shape

(1800, 1)

In [105]:
### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


### Model Training excluding `PoS` features only

### Naive Bayes Training

In [106]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

### SVM Training

In [107]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

### BLRR Training

In [108]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Prediction excluding `pos` only

In [109]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_test, y_predNB)

print(cm)

[[ 4  2  0  1  0  0]
 [ 6  8  7  1  0  1]
 [ 5 19 97 34  2  3]
 [ 0  4 64 72 11  5]
 [ 0  0  1  4  2  6]
 [ 0  0  0  0  1  0]]


In [110]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

#cm = confusion_matrix(y_test, y_predSVML)
#np.set_printoptions(threshold=np.inf)
#print(cm)

In [111]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

#cm = confusion_matrix(y_test, y_predBLRRL)
#print(cm)

In [112]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
#results.head(5)

In [113]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,6.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,4.0,3.0
4,4.0,4.0,4.0,5.0,4.0


## Evaluation excluding `pos` only

In [114]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


### NB

In [115]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.27      0.57      0.36         7
         2.0       0.24      0.35      0.29        23
         3.0       0.57      0.61      0.59       160
         4.0       0.64      0.46      0.54       156
         5.0       0.12      0.15      0.14        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.51       360
   macro avg       0.31      0.36      0.32       360
weighted avg       0.56      0.51      0.53       360

0.5109078114004222


### SVM

In [116]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.53      0.39      0.45        23
         3.0       0.63      0.74      0.68       160
         4.0       0.66      0.65      0.65       156
         5.0       1.00      0.15      0.27        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.47      0.32      0.34       360
weighted avg       0.64      0.64      0.63       360

0.582941571524513


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BLRR

In [117]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.50      0.39      0.44        23
         3.0       0.64      0.77      0.70       160
         4.0       0.70      0.64      0.67       156
         5.0       0.62      0.38      0.48        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.66       360
   macro avg       0.41      0.36      0.38       360
weighted avg       0.64      0.66      0.65       360

0.6172680412371134


### Ensemble

In [118]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.53      0.39      0.45        23
         3.0       0.63      0.76      0.69       160
         4.0       0.69      0.63      0.66       156
         5.0       0.71      0.38      0.50        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.43      0.36      0.38       360
weighted avg       0.64      0.65      0.64       360

0.6037698930341768


#### QWK Scores result excluding `PoS` features only:

Naive Bayes: 0.5109078114004222

SVM: 0.582941571524513

BLRR: 0.6172680412371134

Ensemble: 0.6037698930341768


## `BoW` influence test 

In [119]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

excluding only `BoW` features.

Including chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence, prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words, POS, POS/total_words

In [120]:
X = dataset.iloc[:,1:13].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[2.63900000e+03, 5.27000000e+02, 1.50000000e+01, ...,
        4.17457306e-01, 1.12000000e+02, 2.12523719e-01],
       [8.41000000e+02, 1.80000000e+02, 5.00000000e+00, ...,
        4.55555556e-01, 6.60000000e+01, 3.66666667e-01],
       [1.18100000e+03, 2.61000000e+02, 1.20000000e+01, ...,
        5.51724138e-01, 8.30000000e+01, 3.18007663e-01],
       ...,
       [6.05000000e+02, 1.32000000e+02, 7.00000000e+00, ...,
        5.37878788e-01, 3.40000000e+01, 2.57575758e-01],
       [2.73700000e+03, 5.61000000e+02, 2.20000000e+01, ...,
        5.36541889e-01, 1.68000000e+02, 2.99465241e-01],
       [2.46200000e+03, 5.20000000e+02, 8.00000000e+00, ...,
        4.44230769e-01, 1.65000000e+02, 3.17307692e-01]])

In [121]:
X.shape

y = np.array(y).reshape(-1,1)
y.shape

### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


### Model Training excluding `BoW` features only

### NB

In [122]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

### SVM

In [123]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

### BLRR

In [124]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Prediction excluding `BoW` only

In [125]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_test, y_predNB)

print(cm)

[[  3   3   0   1   0   0]
 [  1   8  11   3   0   0]
 [  0  16 100  42   0   2]
 [  0   4  48  85  15   4]
 [  0   0   0   5   7   1]
 [  0   0   0   0   1   0]]


In [126]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

cm = confusion_matrix(y_test, y_predSVM)
#np.set_printoptions(threshold=np.inf)
print(cm)

[[  0   6   1   0   0   0]
 [  0  11  11   1   0   0]
 [  0   2 115  43   0   0]
 [  0   0  47 109   0   0]
 [  0   0   1  10   2   0]
 [  0   0   0   1   0   0]]


In [127]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

cm = confusion_matrix(y_test, y_predBLRR)
print(cm)

[[  0   4   3   0   0   0]
 [  0   8  14   1   0   0]
 [  0   3 129  27   1   0]
 [  0   0  60  95   1   0]
 [  0   0   0   6   6   1]
 [  0   0   0   0   1   0]]


In [128]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NB,SVM,BLRR
0,3.0,4.0,4.0,5.0
1,5.0,6.0,4.0,4.0
2,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,3.0
4,4.0,5.0,4.0,4.0


In [129]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,4.0,4.0,5.0,4.0
1,5.0,6.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,3.0,3.0
4,4.0,5.0,4.0,4.0,4.0


## Evaluation excluding `BoW` 

In [130]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


### NB

In [131]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.75      0.43      0.55         7
         2.0       0.26      0.35      0.30        23
         3.0       0.63      0.62      0.63       160
         4.0       0.62      0.54      0.58       156
         5.0       0.30      0.54      0.39        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.56       360
   macro avg       0.43      0.41      0.41       360
weighted avg       0.59      0.56      0.57       360

0.5457975662673333


### SVM

In [132]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.58      0.48      0.52        23
         3.0       0.66      0.72      0.69       160
         4.0       0.66      0.70      0.68       156
         5.0       1.00      0.15      0.27        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.66       360
   macro avg       0.48      0.34      0.36       360
weighted avg       0.65      0.66      0.64       360

0.5987980451723682


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BLRR

In [133]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.53      0.35      0.42        23
         3.0       0.63      0.81      0.70       160
         4.0       0.74      0.61      0.67       156
         5.0       0.67      0.46      0.55        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.66       360
   macro avg       0.43      0.37      0.39       360
weighted avg       0.66      0.66      0.65       360

0.6040462427745665


### Ensemble

In [134]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.53      0.39      0.45        23
         3.0       0.66      0.76      0.71       160
         4.0       0.71      0.69      0.70       156
         5.0       0.83      0.38      0.53        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.68       360
   macro avg       0.46      0.37      0.40       360
weighted avg       0.66      0.68      0.67       360

0.6324970599764799



#### QWK Scores result without `BoW` features :

Naive Bayes: 0.5457975662673333

SVM: 0.5987980451723682

BLRR: 0.6040462427745665

Ensemble: 0.6324970599764799


## `Prompt` influence test 

In [135]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [136]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559,4
1,1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210,1
2,2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285,2
3,3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528,4
4,4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,1339.0,286.0,0.0,34.0,11.0,4.681818,31.891253,0.111508,114.0,0.398601,78.0,0.272727,303,293,3
1796,1796,1373.0,282.0,15.0,8.0,15.0,4.868794,12.757794,0.045240,152.0,0.539007,80.0,0.283688,339,333,3
1797,1797,605.0,132.0,7.0,2.0,5.0,4.583333,3.387097,0.025660,71.0,0.537879,34.0,0.257576,164,160,2
1798,1798,2737.0,561.0,22.0,5.0,27.0,4.878788,20.071942,0.035779,301.0,0.536542,168.0,0.299465,542,512,3


excluding only `Prompt` features.

Including chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence, unstemmed, stemmed,POS, POS/total_words

In [137]:
X = dataset.iloc[:,[1,2,3,4,5,6,7,8,13,14]].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X[0]

array([2.63900000e+03, 5.27000000e+02, 1.50000000e+01, 1.30000000e+01,
       2.10000000e+01, 5.00759013e+00, 3.27915870e+01, 6.22231252e-02,
       5.84000000e+02, 5.59000000e+02])

In [138]:
X.shape

y = np.array(y).reshape(-1,1)
y.shape

### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


### Model Training excluding `Prompt` features only

In [139]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

In [140]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

In [141]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Predict excluding `Prompt` features

In [142]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_test, y_predNB)

print(cm)

[[  3   3   0   1   0   0]
 [  6   7   8   1   0   1]
 [  2  21 101  30   2   4]
 [  0   7  59  74   9   7]
 [  0   0   0   6   2   5]
 [  0   0   0   0   1   0]]


In [143]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

cm = confusion_matrix(y_test, y_predSVM)
#np.set_printoptions(threshold=np.inf)
print(cm)

[[  0   5   2   0   0   0]
 [  0   9  13   1   0   0]
 [  0   2 123  35   0   0]
 [  0   0  42 114   0   0]
 [  0   0   0  11   2   0]
 [  0   0   0   0   1   0]]


In [144]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

cm = confusion_matrix(y_test, y_predBLRR)
print(cm)

[[  0   6   1   0   0   0]
 [  0   9  14   0   0   0]
 [  0   5 125  30   0   0]
 [  0   0  53 102   1   0]
 [  0   0   0   6   7   0]
 [  0   0   0   0   1   0]]


In [145]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NB,SVM,BLRR
0,3.0,6.0,4.0,4.0
1,5.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0
3,4.0,3.0,4.0,4.0
4,4.0,4.0,4.0,5.0


In [146]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,6.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,3.0,4.0,4.0,4.0
4,4.0,4.0,4.0,5.0,4.0


## Evaluation Excluding `Prompt` features

In [147]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


## NB

In [148]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.27      0.43      0.33         7
         2.0       0.18      0.30      0.23        23
         3.0       0.60      0.63      0.62       160
         4.0       0.66      0.47      0.55       156
         5.0       0.14      0.15      0.15        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.52       360
   macro avg       0.31      0.33      0.31       360
weighted avg       0.58      0.52      0.54       360

0.49404292251854187


### SVM

In [149]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.56      0.39      0.46        23
         3.0       0.68      0.77      0.72       160
         4.0       0.71      0.73      0.72       156
         5.0       0.67      0.15      0.25        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.69       360
   macro avg       0.44      0.34      0.36       360
weighted avg       0.67      0.69      0.67       360

0.6364668179997329


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BLRR

In [150]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.45      0.39      0.42        23
         3.0       0.65      0.78      0.71       160
         4.0       0.74      0.65      0.69       156
         5.0       0.78      0.54      0.64        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.68       360
   macro avg       0.44      0.39      0.41       360
weighted avg       0.66      0.68      0.67       360

0.6573169183906586


### Ensemble

In [151]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.44      0.35      0.39        23
         3.0       0.66      0.78      0.71       160
         4.0       0.73      0.69      0.71       156
         5.0       0.86      0.46      0.60        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.68       360
   macro avg       0.45      0.38      0.40       360
weighted avg       0.67      0.68      0.67       360

0.6562216755146963


#### QWK Scores result with full features:

Naive Bayes: 0.5172740179839092

SVM: 0.6011377166291838

BLRR: 0.6261591962905719

Ensemble: 0.6060822202049021
#### QWK Scores result with `Prompt` features only:

Naive Bayes: 0.49404292251854187

SVM: 0.6364668179997329

BLRR: 0.6573169183906586

Ensemble: 0.6562216755146963

#### Conclusion

Without `Prompt` features, the QWK score for NB dropped by a small margin but BLRR and SVM improve in a small margin.


### Final Conclusion - Influence
Below are a table conclude the result of model trained by excluding features


| Excluded Features | Naive Bayes| SVM | BLRR | Ensemble | 
| --- | --- | --- | --- | --- | 
| Length | 0.444 | 0.565 | 0.601 | 0.561 | 
| PoS | 0.511 | 0.583 | 0.617 | 0.604 | 
| BoW | 0.546 | 0.599 | 0.604 | 0.632 | 
| Prompt | 0.494 | 0.636 | 0.657 | 0.656 | 
| All features | 0.517 | 0.601 | 0.626 | 0.606 |


Differences between all features and excluded features

| Excluded Features | Naive Bayes| SVM | BLRR | Ensemble | Total Diff (absolute value) | Total Diff |
| --- | --- | --- | --- | --- | --- | --- |
| Length | 0.073 | 0.036 | 0.025 | 0.045 | 0.179 | 0.179 |
| PoS | 0.006 | 0.018 | 0.009 | 0.002 | 0.035 | 0.035 |
| BoW | -0.029 | 0.002 | 0.022 | -0.026 | 0.079 | -0.031 |
| Prompt | 0.023 | -0.035 | -0.031 | -0.050 | 0.139 | -0.093 |


Lower in differences should mean lower in influence on the model.

`Length` has the most influence on Naive Bayes, SVM Model.

`Prompt` has overfitting in SVM and BLRR model

`PoS` has the least influence among all model

`BoW` overfitting NB model